<a href="https://colab.research.google.com/github/Maxibrionest/FMY-Fundamentos-Machine-Learning/blob/main/Entrega4/Entrega4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


In [42]:
url = 'https://raw.githubusercontent.com/Maxibrionest/FMY-Fundamentos-Machine-Learning/main/Entrega1/boston_housing.csv'
df = pd.read_csv(url, sep=",")

In [43]:
def clean_custom(df, col, iqrFences):
    q1 = df[col].quantile(.25)
    q3 = df[col].quantile(.75)
    iqr = q3-q1 #IQR definido anteriormente
    fence_low  = q1 - iqrFences * iqr
    fence_high = q3 + iqrFences * iqr
    df_out = df.loc[(df[col] > fence_low) & (df[col] < fence_high)]
    size = df_out.shape[0]
    pctg = size/df.shape[0]   # calcula el porcentaje de observaciones consideradas respecto al df original
    return df_out, pctg

def clean_outliers_hard(df, col):
    # elimina outliers fuera del rango +- 1.5 IQR de los cuartiles
    # En el boxplot, elimina los datos atípicos
    q1 = df[col].quantile(.25)
    q3 = df[col].quantile(.75)
    iqr = q3-q1 #IQR definido anteriormente
    fence_low  = q1 - 1.5*iqr
    fence_high = q3 + 1.5*iqr
    df_out = df.loc[(df[col] > fence_low) & (df[col] < fence_high)]
    return df_out

def clean_outliers_soft(df, col):
    # elimina outliers fuera del rango +- 3.0 IQR de los cuartiles
    # Dado que sus limites son mas amplios, elimina menos datos o equivalentemente considera mas datos
    q1 = df[col].quantile(.25)
    q3 = df[col].quantile(.75)
    iqr = q3-q1 #IQR definido anteriormente
    fence_low  = q1 - 3*iqr
    fence_high = q3 + 3*iqr
    df_out = df.loc[(df[col] > fence_low) & (df[col] < fence_high)]
    return df_out


In [44]:
df_clean, df_clean_p = clean_custom(df, "crim", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "zn", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "indus", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "nox", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "rm", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "age", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "dis", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "rad", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "tax", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "ptratio", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "black", 1.5)
df_clean, df_clean_p = clean_custom(df_clean, "lstat", 1.5)

In [45]:
corrdat = df_clean.corr()
def getCorr(dat, lim): #obtiene los features con mayor impacto en el target
    feature = []
    valor = []
    
    for i , index in enumerate(dat.index): #bucle para todos los indices
        if abs(dat[index]) > lim:
            feature.append(index)
            #print(index)
            valor.append(dat[index])

    df = pd.DataFrame(data = valor, index = feature, columns = ['corr value'])

    return df

lim = 0.4
corr_value = getCorr(corrdat["medv"], lim)

In [46]:
df_op = df_clean[corr_value.index]

In [47]:
X = df_op.drop(['medv'], axis=1)
Y = df_op.medv.copy()

In [48]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size=0.2, random_state=16062021)

In [49]:
model_tree_grid = DecisionTreeRegressor()

params = {"max_depth": np.arange(3, 10),          # ej: [3,4,5]
          "min_samples_split": np.arange(2, 10),  # ej: [2,3,4]
          "random_state": np.array([16062021])
          }

In [50]:
grid = GridSearchCV(estimator=model_tree_grid, param_grid=params, cv=10, scoring='r2')
grid.fit(Xtrain, Ytrain)

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': array([3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9]),
       

In [51]:
print(grid.best_score_)
print(grid.best_params_)

0.6673883614425868
{'max_depth': 9, 'min_samples_split': 9, 'random_state': 16062021}


In [53]:
model_forest = DecisionTreeRegressor(max_depth=9, min_samples_split=9, random_state=16062021)
model_forest.fit(X, Y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=9,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=16062021, splitter='best')